In [5]:
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def adjust_bounding_box(bbox, original_size, flip_h=False, flip_v=False):
    xmin, ymin, xmax, ymax = bbox
    orig_w, orig_h = original_size

    if flip_h:
        xmin, xmax = orig_w - xmax, orig_w - xmin

    if flip_v:
        ymin, ymax = orig_h - ymax, orig_h - ymin

    return xmin, ymin, xmax, ymax

def make_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Baca data dari CSV lama
df = pd.read_csv("../data/data_original/Pengenalan Aksara Jawa-tensorflow/train/_annotations.csv")

# Daftar untuk menyimpan data yang telah diubah
new_data = []

# Direktori untuk menyimpan gambar dan label hasil augmentasi
original_image_dir = "../data/data_original/Pengenalan Aksara Jawa-tensorflow/train/"
augmented_image_dir = "../data/data_preprocessing/v2.6/"
augmented_label_file = "../data/data_preprocessing/v2.6/augmented_labels.csv"
make_directory(augmented_image_dir)

# ImageDataGenerator untuk augmentasi
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

# Augmentasi setiap kelas sebanyak 5 kali
augmentations_per_class = 300

for class_name in df['class'].unique():
    class_df = df[df['class'] == class_name]
    
    # Buat subfolder untuk kelas ini
    class_dir = os.path.join(augmented_image_dir, class_name)
    make_directory(class_dir)
    
    augment_count = 0  # Tambahkan counter untuk menghitung jumlah augmentasi per kelas
    
    for index, row in class_df.iterrows():
        if augment_count >= augmentations_per_class:
            break
        
        filename = row['filename']
        orig_width = row['width']
        orig_height = row['height']
        class_name = row['class']
        xmin = row['xmin']
        ymin = row['ymin']
        xmax = row['xmax']
        ymax = row['ymax']

        image_path = os.path.join(original_image_dir, filename)
        image = cv2.imread(image_path)
        original_size = (orig_width, orig_height)

        # Convert image to array and expand dimensions
        image_array = np.expand_dims(image, 0)

        # Augmentasi gambar
        for batch in datagen.flow(image_array, batch_size=1):
            aug_image = batch[0].astype('uint8')

            # Adjust bounding box
            flip_h = 'horizontal_flip' in datagen.__dict__ and datagen.horizontal_flip
            flip_v = 'vertical_flip' in datagen.__dict__ and datagen.vertical_flip
            new_bbox = adjust_bounding_box((xmin, ymin, xmax, ymax), original_size, flip_h, flip_v)

            # Simpan gambar yang telah diubah
            aug_filename = f'{filename.split(".")[0]}_aug_{augment_count}.png'
            aug_image_path = os.path.join(class_dir, aug_filename)
            cv2.imwrite(aug_image_path, aug_image)

            # Tambahkan data baru ke daftar
            new_data.append([aug_filename, orig_width, orig_height, class_name, *new_bbox])

            augment_count += 1  # Tingkatkan counter
            if augment_count >= augmentations_per_class:
                break  # keluar dari loop augmentasi jika sudah cukup

# Membuat DataFrame baru dari data yang telah diubah
new_df = pd.DataFrame(new_data, columns=['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])

# Menyimpan DataFrame ke CSV baru
new_df.to_csv(augmented_label_file, index=False)

print("Data bounding box yang telah diubah telah disimpan ke CSV baru dan gambar disimpan ke subfolder berdasarkan kelas.")

Data bounding box yang telah diubah telah disimpan ke CSV baru dan gambar disimpan ke subfolder berdasarkan kelas.


In [6]:
# Lokasi direktori dataset asli
dataset_path = augmented_image_dir

# Daftar kelas (nama subfolder)
classes = os.listdir(dataset_path)

# Dictionary untuk menyimpan jumlah gambar dalam setiap kelas
class_counts = {}

# Iterasi melalui setiap kelas
for class_name in classes:
    class_folder = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_folder):
        # Hitung jumlah file gambar dalam subfolder (kelas)
        num_images = len([name for name in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, name))])
        class_counts[class_name] = num_images

# Tampilkan jumlah gambar dalam setiap kelas
for class_name, count in class_counts.items():
    print(f"Kelas {class_name}: {count} gambar")

Kelas Ba: 300 gambar
Kelas Ca: 300 gambar
Kelas Da: 300 gambar
Kelas Dha: 300 gambar
Kelas Ga: 300 gambar
Kelas Ha: 300 gambar
Kelas Ja: 300 gambar
Kelas Ka: 300 gambar
Kelas La: 300 gambar
Kelas Ma: 300 gambar
Kelas Na: 300 gambar
Kelas Nga: 300 gambar
Kelas Nya: 300 gambar
Kelas Pa: 300 gambar
Kelas Ra: 300 gambar
Kelas Sa: 300 gambar
Kelas Ta: 300 gambar
Kelas Tha: 300 gambar
Kelas Wa: 300 gambar
Kelas Ya: 300 gambar
